In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from astropy.table import Table
from itertools import combinations
from astropy.coordinates import SkyCoord
from astropy import units as u


save = False

sns.set(style="whitegrid")

## Let's try with our data

In [2]:
#Gaia_df = pd.read_csv('DATA/My_Pleiades_filtered_Gaia_DR3_with_mass.csv')
binaries_candidates_df = pd.read_csv('DATA/My_Pleiades_binary_pairs.csv')

binaries_candidates_df.head(20)

,primary_source_id,secondary_source_id
0,66507469798631936,66507469798632320
1,68364544933829376,68364544935515392
2,64956123313498368,64956127609464320
3,66733552578791296,66733556873061120
4,65207709611941376,65207709613871744
5,66798496781121792,66798526845337344
6,65241313435901568,65241313437941504
7,65247704349267584,65248460263511552
8,65266494828710400,65266499126062080
9,65272817023559040,65272821318002560


In [3]:
# taking the binaries candidates from the Gaia DR3
#binaries_candidates_index1 = binaries_candidates_df['primary_source_id'].values
#binaries_candidates_index2 = binaries_candidates_df['secondary_source_id'].values

# Get the Gaia DR3 data for the binary candidates
binaries_1 = pd.read_csv('DATA/My_primary_members_binary_Pleiades_filtered.csv')
binaries_2 = pd.read_csv('DATA/My_secondary_members_binary_Pleiades_filtered.csv')

# orbits parameters
orbit_params = ['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity', 'Mass (M_sun)', 'ra_error', 'dec_error', 'pmra_error', 'pmdec_error', 'parallax_error', 'radial_velocity_error']

# Get the orbit parameters for the binary candidates
binaries_members_1 = binaries_1[orbit_params].copy()
binaries_members_2 = binaries_2[orbit_params].copy()

print(binaries_members_1.shape)
print(binaries_members_2.shape)

print('Check if the binaries candidates df contains Nan values:')
print(binaries_members_1.isnull().values.any())
print(binaries_members_2.isnull().values.any())
print(binaries_members_1.isna().values.any())
print(binaries_members_2.isna().values.any())

# Display the first few rows of the binaries members DataFrame

binaries_members_1.head()

(10, 14)
(10, 14)
Check if the binaries candidates df contains Nan values:
False
False
False
False


,source_id,ra,dec,pmra,pmdec,parallax,radial_velocity,Mass (M_sun),ra_error,dec_error,pmra_error,pmdec_error,parallax_error,radial_velocity_error
0,66507469798631936,57.491184,23.847947,21.026666,-48.132843,7.361154,4.707757,5.001,0.033703,0.018977,0.038665,0.025343,0.033577,0.533899
1,68364544933829376,55.515926,24.712536,21.721589,-45.292485,7.485806,12.204306,0.600,0.042979,0.035992,0.067268,0.057571,0.057112,4.892858
2,64956123313498368,56.453032,23.147851,19.029377,-45.463108,7.404870,5.299024,5.001,0.018650,0.011918,0.024697,0.014803,0.018652,0.219587
3,66733552578791296,56.655542,24.343339,22.380244,-45.516188,7.410337,6.865205,0.750,0.063617,0.040097,0.108746,0.062642,0.060509,3.227011
4,65207709611941376,56.851821,23.914478,20.049108,-44.132590,7.220987,-1.856786,2.600,0.030519,0.022987,0.041599,0.029272,0.034383,2.441681


## Monte Carlo sampling

Since we have only one measurament we apply a MCM sampling to retrive the values of each orbit parameters and its error

| Parameter       | Range Chosen       | Why                                                                                 |
| --------------- | ------------------ | ----------------------------------------------------------------------------------- |
| `parallax`      | (0, 20) mas        | Pleiades parallax is \~7.4 mas. 20 mas is generous and avoids unphysical negatives. |
| `pmra`, `pmdec` | (-100, 100) mas/yr | Wide enough to include all reasonable proper motions for Gaia stars                 |
| `rv`            | (-100, 100) km/s   | Wide enough for almost all stellar RVs in the Galaxy                                |


In [4]:
print(16 * ((300 - int(300 * 0.2)) // 10))

384


In [5]:
import emcee
from collections import deque

# number_of_samples = nwalkers * ((nsteps - int(nsteps * 0.2)) // 10)

np.random.seed(42)  # For reproducibility

# Define MCMC components
def log_prior(theta, obs, obs_err):
    ra, dec, plx, pmra, pmdec, rv = theta
    if 0 < plx < 20 and -100 < pmra < 100 and -100 < pmdec < 100 and -100 < rv < 100:
        return 0.0
    return -np.inf

def log_likelihood(theta, obs, obs_err):
    return -0.5 * np.sum(((theta - obs) / obs_err) ** 2)

def log_posterior(theta, obs, obs_err):
    lp = log_prior(theta, obs, obs_err)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, obs, obs_err)

def mcmc_sample_star(obs_vals, obs_errs, nwalkers=16, nsteps=300):
    ndim = len(obs_vals)
    p0 = obs_vals + 1e-4 * np.random.randn(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(obs_vals, obs_errs))
    sampler.run_mcmc(p0, nsteps, progress=False)
    samples = sampler.get_chain(discard=int(nsteps * 0.2), thin=10, flat=True)
    return samples

def summarize_samples(samples):
    param_names = ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity']
    summary = {}
    for i, name in enumerate(param_names):
        summary[f'{name}'] = np.mean(samples[:, i])

    return summary


def run_MCM_on_binaries(df, list, summary_list):
    for _, row in df.iterrows():
        obs = np.array([
            row['ra'], row['dec'], row['parallax'],
            row['pmra'], row['pmdec'], row['radial_velocity']
        ])
        obs_err = np.array([
            row['ra_error'], row['dec_error'], row['parallax_error'],
            row['pmra_error'], row['pmdec_error'], row['radial_velocity_error']
        ])
        samples = mcmc_sample_star(obs, obs_err, nwalkers=16, nsteps=1000)
        list.append(samples)
        summary = summarize_samples(samples)
        summary_list.append(summary)
    return list, summary_list

# Create summary DataFrame both for binaries_members_1 and binaries_members_2
list1 = deque()
summary_list1 = deque()
list1, summary_list1 = run_MCM_on_binaries(binaries_members_1, list1, summary_list1)

list2 = deque()
summary_list2 = deque()
list2, summary_list2 = run_MCM_on_binaries(binaries_members_2, list2, summary_list2)

# Convert the summary list to a DataFrame
binaries_members_1_MCM = pd.DataFrame(summary_list1)
binaries_members_2_MCM = pd.DataFrame(summary_list2)

list1 = np.transpose(list1, axes=(1, 0, 2))
list2 = np.transpose(list2, axes=(1, 0, 2))

print(list1.shape)
print(list1[0])
binaries_members_1_MCM.head(10)


(1280, 10, 6)
[[ 57.51784956  23.82789079   7.30916736  21.05059524 -48.14488969
    4.18721844]
 [ 55.4854542   24.70344112   7.59408836  21.79872386 -45.32497791
   14.11703199]
 [ 56.42410906  23.14369709   7.38237191  19.02626182 -45.48718058
    5.43590476]
 [ 56.6716448   24.32803791   7.33265269  22.3091596  -45.53492611
    7.98601181]
 [ 56.86793172  23.9136751    7.22331311  20.05119219 -44.13321199
   -1.37887259]
 [ 56.47079987  24.54027515   7.23190617  20.39215152 -46.02012183
   11.27002387]
 [ 56.16373865  23.87907422   7.365172    19.5590898  -45.44174775
   16.45879446]
 [ 56.05566467  24.01710125   7.60190663  21.4574038  -45.88420629
    4.15955001]
 [ 55.86407296  24.2070358    7.24627112  20.35790047 -46.51889798
   19.01890947]
 [ 56.02094444  24.13102277   7.30966998  20.31770699 -47.80654243
   11.63358028]]


,ra,dec,parallax,pmra,pmdec,radial_velocity
0,57.491466,23.848500,7.363171,21.021351,-48.133672,4.687575
1,55.520199,24.713419,7.487410,21.729241,-45.293407,12.071893
2,56.452669,23.147772,7.406736,19.030893,-45.463276,5.291427
3,56.657712,24.343744,7.403738,22.382387,-45.516553,6.960689
4,56.850564,23.915247,7.221184,20.048630,-44.129799,-1.524162
5,56.477196,24.551208,7.303484,20.331635,-46.013909,11.756057
6,56.133926,23.874283,7.365840,19.582492,-45.428033,16.332646
7,56.053716,24.031373,7.594565,21.404246,-45.884287,4.144967
8,55.902495,24.226303,7.313981,20.326572,-46.512418,-0.837315
9,56.097853,24.132501,7.255867,20.320970,-47.789202,10.145180


In [10]:
print(list1[0])
print(list1[1])

[[ 57.51784956  23.82789079   7.30916736  21.05059524 -48.14488969
    4.18721844]
 [ 55.4854542   24.70344112   7.59408836  21.79872386 -45.32497791
   14.11703199]
 [ 56.42410906  23.14369709   7.38237191  19.02626182 -45.48718058
    5.43590476]
 [ 56.6716448   24.32803791   7.33265269  22.3091596  -45.53492611
    7.98601181]
 [ 56.86793172  23.9136751    7.22331311  20.05119219 -44.13321199
   -1.37887259]
 [ 56.47079987  24.54027515   7.23190617  20.39215152 -46.02012183
   11.27002387]
 [ 56.16373865  23.87907422   7.365172    19.5590898  -45.44174775
   16.45879446]
 [ 56.05566467  24.01710125   7.60190663  21.4574038  -45.88420629
    4.15955001]
 [ 55.86407296  24.2070358    7.24627112  20.35790047 -46.51889798
   19.01890947]
 [ 56.02094444  24.13102277   7.30966998  20.31770699 -47.80654243
   11.63358028]]
[[ 57.49578697  23.82878628   7.32052999  21.02030556 -48.0855157
    4.12310311]
 [ 55.52709165  24.7322475    7.46019658  21.63994809 -45.31243068
   10.14233821]
 [ 5

In [6]:
## adding the corresponding source_id to the MCM results
binaries_members_1_MCM.insert(0, 'source_id', binaries_members_1['source_id'].values)
binaries_members_2_MCM.insert(0, 'source_id', binaries_members_2['source_id'].values)

#binaries_members_1_MCM.insert(1, 'Name', binaries_members_1['Name'].values)
#binaries_members_2_MCM.insert(1, 'Name', binaries_members_2['Name'].values)

binaries_members_1_MCM.insert(1, 'mass', binaries_members_1['Mass (M_sun)'].values)
binaries_members_2_MCM.insert(1, 'mass', binaries_members_2['Mass (M_sun)'].values)

binaries_members_2_MCM.head(10)

,source_id,mass,ra,dec,parallax,pmra,pmdec,radial_velocity
0,66507469798632320,1.067,57.523066,23.843829,7.666924,21.529037,-43.576213,1.119084
1,68364544935515392,0.614,55.507484,24.726694,7.805989,22.195642,-46.619399,5.352550
2,64956127609464320,2.801,56.454640,23.147303,7.418069,20.517538,-45.699710,7.721754
3,66733556873061120,5.001,56.660814,24.342126,7.294581,16.387006,-44.770694,4.186389
4,65207709613871744,1.163,56.853242,23.909874,7.193719,19.610298,-46.330140,5.340594
5,66798526845337344,0.625,56.481867,24.553933,7.355720,19.536592,-45.548849,4.438023
6,65241313437941504,0.473,56.137939,23.878460,7.355039,19.612429,-46.532992,10.174809
7,65248460263511552,0.809,56.056028,24.035772,7.355366,20.135138,-45.155576,4.713042
8,65266499126062080,5.001,55.907287,24.226607,7.711384,19.558893,-42.105485,-90.636398
9,65272821318002560,1.395,56.101617,24.138580,7.375303,28.074611,-43.090645,6.329138


In [7]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

def compute_orbital_elements_from_phase_space(star1, star2, m1, m2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # Average parallax to enforce consistent distance scale
    avg_parallax = (star1[2] + star2[2]) / 2  # mas
    distance = (1000 / avg_parallax) * u.pc

    # SkyCoord objects
    c1 = SkyCoord(
        ra=star1[0] * u.deg,
        dec=star1[1] * u.deg,
        distance=distance,
        #distance = (1000 / star1['parallax']) * u.pc,
        pm_ra_cosdec=star1[3] * u.mas/u.yr,
        pm_dec=star1[4] * u.mas/u.yr,
        #radial_velocity=star1[5] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2[0] * u.deg,
        dec=star2[1] * u.deg,
        distance=distance,
        #distance = (1000 / star2['parallax']) * u.pc,
        pm_ra_cosdec=star2[3] * u.mas/u.yr,
        pm_dec=star2[4] * u.mas/u.yr,
        #radial_velocity=star2[5] * u.km/u.s
    )

    # Masses and CM quantities
    #m1, m2 = star1['mass'], star2['mass']
    M = m1 + m2
    M_r = (m1 * m2) / M
    mu = G*M

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value

    # Relative position and velocity in CM frame
    r_rel = r2 - r1
    v_rel = v2 - v1
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)


    # Angular momentum
    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)


    # Specific mechanical energy
    E = 0.5*v_mag**2. - mu/r_mag
    E_total = E * M_r  # Total energy in the system

    a = - mu/(2 * E)  # Semi-major axis


    # Eccentricity vector
    e_vec = (np.cross(v_rel, h_vec) / mu) - (r_rel / r_mag) 
    e = np.linalg.norm(e_vec)

    # Inclination
    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    # Longitude of ascending node
    if n_mag != 0:
        Omega = np.degrees(np.arccos(n_vec[0] / n_mag))
        if n_vec[1] < 0:
            Omega = 360 - Omega
    else:
        Omega = 0

    # Argument of periapsis
    if n_mag != 0 and e > 1e-8:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly
    if e > 1e-8:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    # Orbital period (years)
    T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))

    # Return the orbital elements as a dictionary

    return [a, e, i_deg, Omega, omega, nu, T_yr]


In [8]:
print(list1.shape)

(1280, 10, 6)


In [ ]:
from tqdm import tqdm

if not os.path.exists('results/My_results_wErrors'):
    os.makedirs('results/My_results_wErrors/')


result_orbital_elements_pair = np.empty((list1.shape[0], len(binaries_members_1_MCM), 7))

for t in tqdm(range(list1.shape[0]), desc="Processing MCM iterations"):
    # For each MCM iteration, we will compute the orbital elements for each pair of stars
    binaries_members_1_MCM['mass'] = binaries_members_1_MCM['mass'].astype(float)
    binaries_members_2_MCM['mass'] = binaries_members_2_MCM['mass'].astype(float)
    
    # Convert the lists to numpy arrays for easier indexing
    list1[t] = np.array(list1[t])
    list2[t] = np.array(list2[t])
    # For each MCM iteration, we will compute the orbital elements for each pair of stars
    binaries_members_1_MCM['mass'] = binaries_members_1_MCM['mass'].astype(float)
    binaries_members_2_MCM['mass'] = binaries_members_2_MCM['mass'].astype(float)
    
    # Initialize the result array for this MCM iteration
    result_orbital_elements_pair[t] = np.zeros((len(binaries_members_1_MCM), 7))
    # Iterate through the pairs of stars in the lists
    for i in range(len(binaries_members_1_MCM)):
        star1 = list1[t, i, :]
        star2 = list2[t, i, :]
        
        m1 = binaries_members_1_MCM.iloc[i]['mass']
        m2 = binaries_members_2_MCM.iloc[i]['mass']
        # Compute orbital elements
        orbital_elements = compute_orbital_elements_from_phase_space(star1, star2, m1, m2)
        
        result_orbital_elements_pair[t,i] = orbital_elements

# Convert the dictionary to a DataFrame
print(result_orbital_elements_pair.shape)

# calculating the mean and std of the orbital elements
orbital_elements_mean = np.mean(result_orbital_elements_pair, axis=0)
orbital_elements_std = np.std(result_orbital_elements_pair, axis=0)

orbital_elements_df = pd.DataFrame({
    'a (pc)': orbital_elements_mean[:,0],
    'e': orbital_elements_mean[:,1],
    'i (deg)': orbital_elements_mean[:,2],
    'Omega (deg)': orbital_elements_mean[:,3],
    'omega (deg)': orbital_elements_mean[:,4],
    'nu (deg)': orbital_elements_mean[:,5],
    'T (yr)': orbital_elements_mean[:,6],
    'a_std (pc)': orbital_elements_std[:,0],
    'e_std': orbital_elements_std[:,1],
    'i_std (deg)': orbital_elements_std[:,2],
    'Omega_std (deg)': orbital_elements_std[:,3],
    'omega_std (deg)': orbital_elements_std[:,4],
    'nu_std (deg)': orbital_elements_std[:,5],
    'T_std (yr)': orbital_elements_std[:,6]
    })

if save:
    # Save the DataFrame to a CSV file
    orbital_elements_df.to_csv('results/My_results/orbital_elements_wRV.csv', index=False)

    # Save the results to CSV files
    binaries_members_1_MCM.to_csv('results/My_results/binaries_members_1_MCM.csv', index=False)
    binaries_members_2_MCM.to_csv('results/My_results/binaries_members_2_MCM.csv', index=False)

orbital_elements_df.head(20)    


Processing MCM iterations:   0%|          | 0/1280 [00:00<?, ?it/s]C:\Users\bosca\AppData\Local\Temp\ipykernel_18188\3076926357.py:101: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
Processing MCM iterations:   0%|          | 1/1280 [00:00<02:56,  7.25it/s]C:\Users\bosca\AppData\Local\Temp\ipykernel_18188\3076926357.py:101: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_18188\3076926357.py:101: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
Processing MCM iterations:   0%|          | 3/1280 [00:00<01:51, 11.44it/s]C:\Users\bosca\AppData\Local\Temp\ipykernel_18188\3076926357.py:101: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_18188\3076926357.py:101: RuntimeWarning: invalid value encountered in sqrt


(1280, 10, 7)


,a (pc),e,i (deg),Omega (deg),omega (deg),nu (deg),T (yr),a_std (pc),e_std,i_std (deg),Omega_std (deg),omega_std (deg),nu_std (deg),T_std (yr)
0,-0.003173,132.804894,87.602182,230.609378,210.104053,173.028647,NaN,0.000289,102.554137,23.752640,87.050061,170.001960,145.702547,NaN
1,-0.007463,44.385588,85.553110,233.379410,173.390330,187.206212,NaN,0.003374,35.604768,24.501150,80.184358,22.501001,135.500645,NaN
2,-0.112145,1.363903,86.673869,233.898745,179.914699,178.912737,NaN,2.680590,0.760001,24.186637,88.811155,99.357889,94.100427,NaN
3,-0.001674,75.465417,89.039217,240.161142,183.615615,183.631516,NaN,0.000133,58.992618,24.898496,89.260904,97.605812,129.364809,NaN
4,-0.009285,12.298103,91.103748,241.049469,177.489657,166.080758,NaN,0.011496,9.546585,23.663326,87.714848,22.966835,133.750117,NaN
5,-0.105005,1.981595,83.239783,232.736315,172.785915,180.490978,NaN,1.185147,1.453348,25.962920,85.056165,117.371069,106.716357,NaN
6,-0.012038,9.930360,90.999512,242.882902,178.497593,183.954841,NaN,0.029737,7.942054,23.655902,85.730927,21.296754,133.083467,NaN
7,-0.009570,18.136329,83.245787,231.320841,170.942823,175.792823,NaN,0.017725,14.284931,25.834624,84.241369,122.086913,133.052731,NaN
8,-0.003085,68.932296,88.914025,234.521754,182.275147,186.284630,NaN,0.000676,53.191106,24.042409,87.287943,169.660481,138.268765,NaN
9,-0.000268,530.224725,91.416643,241.585138,172.321842,171.652317,NaN,0.000008,400.637547,24.082552,89.447812,121.061296,135.528160,NaN
